In [8]:
import re
import numpy as np
import pandas as pd

DATA_PATH = "Gyrate_UserResults.xlsx"
KEEP_ATTEMPT = "first"   # options: "first", "latest", "best"

def parse_failed_levels(s: str | float | None) -> dict[int, int]:
    """
    Parse the 'FailedLevels' column into {level -> total_failures_at_that_level}.
    Each token looks like 'A:B:C'. We keep max(B) per A (usually equals the count).
    """
    fails: dict[int, int] = {}
    if pd.isna(s):
        return fails
    s = str(s).strip()
    if not s:
        return fails
    for tok in s.split(","):
        parts = tok.strip().split(":")
        if len(parts) < 2:
            continue
        a, b = parts[0].strip(), parts[1].strip()
        if a.isdigit() and b.isdigit():
            A = int(a)
            B = int(b)
            fails[A] = max(fails.get(A, 0), B)
    return fails

# test
parse_failed_levels("1:1:0,9:1:6,9:2:4,10:1:6,11:1:1,11:2:6,11:3:5,12:1:0,12:2:6,12:3:5")

df = pd.read_excel(DATA_PATH)
df.columns = [c.strip() for c in df.columns]
df[df['AccountId'] == 163974]


,AccountId,AssessmentId,AssessmentVersionId,Score,DisplayScore,Percentage,Percentile,Locale,CreationDate,TotalResponses,CorrectResponses,ReactionTime,BonusAwards,Level,FailedLevels,EarlyResponses,LateResponses
160,163974,12,151,18,1500,85.714286,0,pt-BR,2025-10-13 15:35:38.542,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
161,163974,12,151,18,1500,85.714286,0,pt-BR,2025-10-13 15:35:38.542,NaN,NaN,14.45375,NaN,NaN,NaN,NaN,NaN
162,163974,12,151,18,1500,85.714286,0,pt-BR,2025-10-13 15:35:38.542,36.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN
163,163974,12,151,18,1500,85.714286,0,pt-BR,2025-10-13 15:35:38.542,NaN,NaN,NaN,0.0,18.0,"6:1:0,6:2:5,6:3:5,7:1:5,12:1:5,12:2:2,14:1:0,1...",NaN,NaN


In [13]:
x = df[df['AccountId'] == 163974]
x["FailedLevels"].iloc[3]

'6:1:0,6:2:5,6:3:5,7:1:5,12:1:5,12:2:2,14:1:0,14:2:0,14:3:0,16:1:11,16:2:5,16:3:0,17:1:1,18:1:3,18:2:7,19:1:6,19:2:11,19:3:0,20:1:0,20:2:13,20:3:8'

In [14]:
def load_sessions_dev(path=DATA_PATH, keep_attempt=KEEP_ATTEMPT):
    """
    Read the raw summary rows and coalesce each group of duplicate-key rows into one row
    by taking the first non-null value inside each group for the data columns.
    """
    df = pd.read_excel(path)
    df.columns = [c.strip() for c in df.columns]

    # Drop unused columns if present
    df = df.drop(columns=[c for c in ["Percentile", "BonusAwards", "EarlyResponses", "LateResponses"] if c in df.columns],
                 errors="ignore")

    # Types we rely on
    for c in ["Level", "CorrectResponses", "TotalResponses", "ReactionTime"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    if "CreationDate" in df.columns:
        df["CreationDate"] = pd.to_datetime(df["CreationDate"], errors="coerce", infer_datetime_format=True)

    # Columns that define identical groups (these are equal within each 4-row block)
    key_cols = [
        "AccountId", "AssessmentId", "AssessmentVersionId", "Score",
        "DisplayScore", "Percentage", "Locale", "CreationDate"
    ]
    key_cols = [c for c in key_cols if c in df.columns]  # guard

    # Columns where only one row in the block has the value; pick the first non-null
    value_cols = [
        "TotalResponses", "CorrectResponses", "ReactionTime", "Level", "FailedLevels"
    ]
    value_cols = [c for c in value_cols if c in df.columns]  # guard

    # Helper: first non-null in the group
    def first_valid(s):
        s = s.dropna()
        return s.iloc[0] if not s.empty else np.nan

    # Group by the invariant columns and coalesce value columns
    if key_cols:
        agg_map = {c: first_valid for c in value_cols}
        collapsed = (
            df.groupby(key_cols, as_index=False)
              .agg(agg_map)  # returns one row per unique key
        )
    else:
        # Fallback if keys missing: just take first non-null per entire df (unlikely)
        collapsed = df[value_cols].agg(first_valid).to_frame().T

    return collapsed


def remove_repeated_attempts(df,  keep_attempt=KEEP_ATTEMPT): 

    """
    Read the raw summary rows and keep ONE row per AccountId.
    - 'first'  : earliest CreationDate per AccountId
    - 'latest' : latest CreationDate per AccountId
    - 'best'   : highest Level, tie-broken by latest time
    """

    if keep_attempt == "first":
        df = (df.sort_values(["AccountId", "CreationDate"], ascending=[True, True])
                .drop_duplicates(subset=["AccountId"], keep="first"))
    elif keep_attempt == "latest":
        df = (df.sort_values(["AccountId", "CreationDate"], ascending=[True, False])
                .drop_duplicates(subset=["AccountId"], keep="first"))
    elif keep_attempt == "best":
        df = (df.sort_values(["AccountId", "Level", "CreationDate"],
                             ascending=[True, False, False])
                .drop_duplicates(subset=["AccountId"], keep="first"))
    else:
        raise ValueError("KEEP_ATTEMPT must be 'first', 'latest', or 'best'.")

    return df




In [15]:
df = load_sessions_dev(DATA_PATH, KEEP_ATTEMPT)
df

/var/folders/ww/rbvxp9wd1r18p36rblrbsz0r0000gp/T/ipykernel_13894/4148836556.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["CreationDate"] = pd.to_datetime(df["CreationDate"], errors="coerce", infer_datetime_format=True)


,AccountId,AssessmentId,AssessmentVersionId,Score,DisplayScore,Percentage,Locale,CreationDate,TotalResponses,CorrectResponses,ReactionTime,Level,FailedLevels
0,130850,12,151,8,800,38.095238,en,2025-10-13 17:02:37.326,18.0,8.0,1.687625,8.0,"1:1:0,7:1:2,7:2:5,8:1:4,9:1:0,9:2:4,9:3:7,10:1..."
1,163974,12,151,18,1500,85.714286,pt-BR,2025-10-13 15:35:38.542,36.0,15.0,14.453750,18.0,"6:1:0,6:2:5,6:3:5,7:1:5,12:1:5,12:2:2,14:1:0,1..."
2,168052,12,151,6,600,28.571429,nl,2025-09-29 12:14:35.498,13.0,6.0,2.271600,6.0,"6:1:5,7:1:4,7:2:3,7:3:0,8:1:6,8:2:2,8:3:4"
3,227411,12,151,21,1800,100.000000,es-MX,2025-09-27 12:43:00.126,36.0,18.0,17.825389,21.0,"1:1:1,7:1:4,7:2:4,7:3:5,8:1:3,8:2:1,13:1:0,15:..."
4,258059,12,151,8,800,38.095238,es-MX,2025-10-10 20:32:58.831,18.0,8.0,2.178375,8.0,"1:1:0,7:1:5,7:2:2,8:1:2,9:1:7,9:2:7,9:3:2,10:1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,694630,12,151,8,800,38.095238,es,2025-10-14 00:54:07.416,14.0,8.0,2.044714,8.0,"9:1:7,9:2:1,9:3:2,10:1:3,10:2:5,10:3:5"
746,694645,12,151,6,600,28.571429,en,2025-10-14 01:45:03.021,15.0,6.0,2.759800,6.0,"4:1:2,6:1:5,6:2:3,7:1:1,7:2:2,7:3:0,8:1:4,8:2:..."
747,694710,12,151,6,500,28.571429,en,2025-10-14 08:43:19.184,16.0,5.0,2.939500,6.0,"4:1:3,4:2:3,4:3:2,5:1:3,6:1:4,7:1:2,7:2:2,7:3:..."
748,694780,12,151,7,700,33.333333,en,2025-10-14 10:09:11.314,19.0,7.0,3.358000,7.0,"4:1:3,4:2:3,6:1:5,6:2:5,7:1:5,7:2:4,8:1:3,8:2:..."


In [16]:
print(len(set(df['AccountId'].tolist())))
df = remove_repeated_attempts(df, KEEP_ATTEMPT)
df



702


,AccountId,AssessmentId,AssessmentVersionId,Score,DisplayScore,Percentage,Locale,CreationDate,TotalResponses,CorrectResponses,ReactionTime,Level,FailedLevels
0,130850,12,151,8,800,38.095238,en,2025-10-13 17:02:37.326,18.0,8.0,1.687625,8.0,"1:1:0,7:1:2,7:2:5,8:1:4,9:1:0,9:2:4,9:3:7,10:1..."
1,163974,12,151,18,1500,85.714286,pt-BR,2025-10-13 15:35:38.542,36.0,15.0,14.453750,18.0,"6:1:0,6:2:5,6:3:5,7:1:5,12:1:5,12:2:2,14:1:0,1..."
2,168052,12,151,6,600,28.571429,nl,2025-09-29 12:14:35.498,13.0,6.0,2.271600,6.0,"6:1:5,7:1:4,7:2:3,7:3:0,8:1:6,8:2:2,8:3:4"
3,227411,12,151,21,1800,100.000000,es-MX,2025-09-27 12:43:00.126,36.0,18.0,17.825389,21.0,"1:1:1,7:1:4,7:2:4,7:3:5,8:1:3,8:2:1,13:1:0,15:..."
4,258059,12,151,8,800,38.095238,es-MX,2025-10-10 20:32:58.831,18.0,8.0,2.178375,8.0,"1:1:0,7:1:5,7:2:2,8:1:2,9:1:7,9:2:7,9:3:2,10:1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,694630,12,151,8,800,38.095238,es,2025-10-14 00:54:07.416,14.0,8.0,2.044714,8.0,"9:1:7,9:2:1,9:3:2,10:1:3,10:2:5,10:3:5"
746,694645,12,151,6,600,28.571429,en,2025-10-14 01:45:03.021,15.0,6.0,2.759800,6.0,"4:1:2,6:1:5,6:2:3,7:1:1,7:2:2,7:3:0,8:1:4,8:2:..."
747,694710,12,151,6,500,28.571429,en,2025-10-14 08:43:19.184,16.0,5.0,2.939500,6.0,"4:1:3,4:2:3,4:3:2,5:1:3,6:1:4,7:1:2,7:2:2,7:3:..."
748,694780,12,151,7,700,33.333333,en,2025-10-14 10:09:11.314,19.0,7.0,3.358000,7.0,"4:1:3,4:2:3,6:1:5,6:2:5,7:1:5,7:2:4,8:1:3,8:2:..."


In [18]:


def _pass_attempts_from_failed_dict(fails: dict[int, int], n_levels: int = 21) -> list[int]:
    """
    Internal helper.
    Returns a list pass_attempt[1..n], where for each level L:
      - 1,2,3 = passed on that attempt (after that many failures)
      - 0     = failed all 3 attempts at that level
      - -1    = not reached (due to earlier two consecutive 3-fails); treated as fail
    """
    # Normalize and clamp failure counts for levels 1..n
    level_fails = [0] * (n_levels + 1)  # index by level
    for L, cnt in fails.items():
        if 1 <= L <= n_levels:
            c = int(cnt)
            if c < 0: c = 0
            if c > 3: c = 3
            level_fails[L] = c

    # Find first occurrence of two consecutive 3-fails (termination point)
    terminate_at = None  # the second level of the two consecutive fails
    for L in range(1, n_levels):
        if level_fails[L] == 3 and level_fails[L + 1] == 3:
            terminate_at = L + 1
            break

    # Build pass attempts
    pass_attempt = [None] * (n_levels + 1)
    for L in range(1, n_levels + 1):
        if terminate_at is not None and L > terminate_at:
            pass_attempt[L] = -1  # not reached; treated as fail
        else:
            f = level_fails[L]
            if f >= 3:
                pass_attempt[L] = 0   # failed all
            else:
                pass_attempt[L] = f + 1  # passed on attempt 1/2/3
    return pass_attempt[1:]  # 1..n


def make_item_response_matrix_binary(
    df: pd.DataFrame,
    id_col: str = "AccountId",
    failed_col: str = "FailedLevels",
    n_levels: int = 21
) -> pd.DataFrame:
    """
    One column per level (21 columns). Entry is:
      - 1 if participant passed the level (any attempt),
      - 0 if participant failed all 3 attempts,
      - 0 if the level was not reached due to two consecutive prior fails.
    Returns a DataFrame indexed by AccountId with columns L1..L{n}.
    """
    rows = []
    ids = []

    for _, row in df[[id_col, failed_col]].iterrows():
        pid = row[id_col]
        fails = parse_failed_levels(row[failed_col])
        attempts = _pass_attempts_from_failed_dict(fails, n_levels=n_levels)
        # Map attempts to binary score
        # 1/2/3 -> 1 (passed), 0 or -1 -> 0 (failed/not reached)
        binary = [1 if a in (1, 2, 3) else 0 for a in attempts]
        rows.append(binary)
        ids.append(pid)

    cols = [f"L{L}" for L in range(1, n_levels + 1)]
    M = pd.DataFrame(rows, index=ids, columns=cols).astype(int)
    return M


def make_item_response_matrix_3attempts(
    df: pd.DataFrame,
    id_col: str = "AccountId",
    failed_col: str = "FailedLevels",
    n_levels: int = 21
) -> pd.DataFrame:
    """
    Three columns per level (total 3*n_levels). For each level:
      - Passed on 1st attempt: [1, 1, 1]
      - Passed on 2nd attempt: [0, 1, 1]
      - Passed on 3rd attempt: [0, 0, 1]
      - Failed all / Not reached: [0, 0, 0]
    Returns a DataFrame indexed by AccountId with columns:
      L1_A1, L1_A2, L1_A3, ..., L{n}_A1, L{n}_A2, L{n}_A3
    """
    rows = []
    ids = []

    for _, row in df[[id_col, failed_col]].iterrows():
        pid = row[id_col]
        fails = parse_failed_levels(row[failed_col])
        attempts = _pass_attempts_from_failed_dict(fails, n_levels=n_levels)

        triple = []
        for a in attempts:
            if a == 1:
                triple.extend([1, 1, 1])
            elif a == 2:
                triple.extend([0, 1, 1])
            elif a == 3:
                triple.extend([0, 0, 1])
            else:  # a == 0 (failed all) or a == -1 (not reached)
                triple.extend([0, 0, 0])
        rows.append(triple)
        ids.append(pid)

    cols = [f"L{L}_A{A}" for L in range(1, n_levels + 1) for A in (1, 2, 3)]
    M = pd.DataFrame(rows, index=ids, columns=cols).astype(int)
    return M


In [20]:
M = make_item_response_matrix_binary(df)
M

,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,...,L12,L13,L14,L15,L16,L17,L18,L19,L20,L21
130850,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
163974,1,1,1,1,1,0,1,1,1,1,...,1,1,0,1,0,1,1,0,0,0
168052,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
227411,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,0,1,0,1,1
258059,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694630,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
694645,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
694710,1,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
694780,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
def test_binary_vs_correct_responses(
    df: pd.DataFrame,
    id_col: str = "AccountId",
    correct_col: str = "CorrectResponses",
    failed_col: str = "FailedLevels",
    n_levels: int = 21
) -> pd.DataFrame:
    """
    For each participant:
      - Compute sum over make_item_response_matrix_binary row.
      - Compare to CorrectResponses.
    Returns a report DataFrame indexed by AccountId with columns:
      ['Computed_Sum', 'CorrectResponses', 'Diff', 'Pass'].
    """
    # Build binary matrix and row sums
    M_bin = make_item_response_matrix_binary(
        df, id_col=id_col, failed_col=failed_col, n_levels=n_levels
    )
    computed = M_bin.sum(axis=1)

    # Prepare S from df
    S = pd.to_numeric(df[correct_col], errors="coerce")
    S_by_id = pd.Series(S.values, index=df[id_col].values, name="CorrectResponses")

    # Align by AccountId
    report = pd.concat(
        [computed.rename("Computed_Sum"), S_by_id], axis=1
    )

    # Compute diagnostics
    report["Diff"] = report["Computed_Sum"] - report["CorrectResponses"]
    report["Pass"] = report["Computed_Sum"] == report["CorrectResponses"]

    return report


def test_attempt_matrix_bounds(
    df: pd.DataFrame,
    id_col: str = "AccountId",
    correct_col: str = "CorrectResponses",
    failed_col: str = "FailedLevels",
    n_levels: int = 21
) -> pd.DataFrame:
    """
    For each participant:
      Let S = CorrectResponses and N = sum of ones in 3-attempt matrix row.
      Check S <= N <= 3S.
    Returns a report DataFrame indexed by AccountId with columns:
      ['S', 'N', 'Lower_OK', 'Upper_OK', 'Pass'].
    """
    # Build 3-attempt matrix and row sums
    M_3 = make_item_response_matrix_3attempts(
        df, id_col=id_col, failed_col=failed_col, n_levels=n_levels
    )
    N = M_3.sum(axis=1).rename("N")

    # Prepare S
    S = pd.to_numeric(df[correct_col], errors="coerce")
    S_by_id = pd.Series(S.values, index=df[id_col].values, name="S")

    # Align and check bounds
    report = pd.concat([S_by_id, N], axis=1)
    report["Lower_OK"] = report["S"] <= report["N"]
    report["Upper_OK"] = report["N"] <= 3 * report["S"]
    report["Pass"] = report["Lower_OK"] & report["Upper_OK"]

    return report

In [22]:
r1 = test_binary_vs_correct_responses(df)
r2 = test_attempt_matrix_bounds(df)
# # Failing rows:
failures_r1 = r1[~r1["Pass"]]
failures_r2 = r2[~r2["Pass"]]
# # Quick summaries:
print("Binary vs CorrectResponses - pass rate:",
       r1["Pass"].mean(), f"({r1['Pass'].sum()} / {len(r1)})")
print("3-attempt bounds - pass rate:",
       r2["Pass"].mean(), f"({r2['Pass'].sum()} / {len(r2)})")

Binary vs CorrectResponses - pass rate: 1.0 (702 / 702)
3-attempt bounds - pass rate: 1.0 (702 / 702)
